In this notebook, we apply GMM-MI to estimate the mutual information between physical parameters and latent variables of the $\beta$-VAE model trained on stellar spectra in [Sedaghat et al. (2021)](https://academic.oup.com/mnras/article-abstract/501/4/6026/6121645?redirectedFrom=fulltext&login=false). We compare our estimates with their results, obtained with histograms as density estimators. The most meaningful latents are found using the median absolute deviation (MAD). All data can be obtained from the [corresponding repository](https://github.com/NimSed/astro-machines).

For this notebook, you will also need `pandas` and `astropy`; you can quickly run form your terminal:
    
    pip install pandas==1.3.4
    pip install pandas==4.3.1

### Import packages

In [1]:
import numpy as np
import yaml
import pandas as pd
from astropy.stats import sigma_clip
from gmm_mi.gmm import GMM
from gmm_mi.mi import EstimateMI
from gmm_mi.utils.plotting import create_heatmap, annotate_heatmap

### Load the data

In [2]:
# latents with highest MAD
significant_dimensions = [11, 19, 58, 85, 99, 124]
stellar_params = ['radvel','Teff','Mass','[M/H]','airmass','snr']

# these steps are taken from the astro-machines repository
codes_file = './astro_machines_data/encoded_harps_unique_128d_e182_i1500000.npy'
codes = pd.DataFrame(np.load(codes_file))
labels = pd.read_csv('./astro_machines_data/harps_metadata_and_labels.csv')

# if looking at the 'unique' subset, drop unnecessary rows
if "unique" in codes_file:
    labels.dropna(subset=['unique_subset'], inplace=True)

# and reindex the codes dataframe accordingly
codes.loc[:,'dp_id'] = labels['dp_id'].values
labels = labels.set_index('dp_id')
codes = codes.set_index('dp_id')

### Calculate MI between significant latents and stellar spectra parameters

In [ ]:
run_this_cell = True

if not run_this_cell:
    print('Not running this cell...')
    pass
else:
    print(f'Estimating MI...')
    all_MI_estimates = np.zeros((len(stellar_params), len(significant_dimensions), 2))
    save = True

    # load parameters from yaml file and assign them to variables
    # not the cleanest and safest way, but it should work
    with open('./config_files/config_parameters_9-11.yml', 'rb') as f:
        conf = yaml.safe_load(f.read())
    locals().update(conf)
    # we need to slightly increase reg_covar to avoid
    # a singularity in very few occasions
    reg_covar = 1e-13
    
    for li, stellar_param in enumerate(stellar_params):
        label = labels.loc[:, stellar_param].values
        ind_mask = ~np.isnan(label)
        clipped_label = sigma_clip(label,sigma=5,masked=True)
        ind_mask = ind_mask & (~clipped_label.mask)
        stellar_samples = clipped_label.data[ind_mask]    

        for zi, latent_id in enumerate(significant_dimensions):
            significant_latent = codes.iloc[:, latent_id].values
            latent = significant_latent[ind_mask]
            X = np.stack((stellar_samples, latent), axis=0).T          
            MI_mean, MI_std = EstimateMI(n_folds=n_folds, n_inits=n_inits, init_type=init_type, reg_covar=reg_covar, 
                     tol=tol, max_iter=max_iter, bootstrap=bootstrap, n_bootstrap=n_bootstrap, 
                     MC_samples=MC_samples, metric_method=metric_method, patience=patience, 
                     verbose=verbose, return_lcurves=return_lcurves).fit(X)   
           
            # store results
            all_MI_estimates[li, zi, 0] = MI_mean
            all_MI_estimates[li, zi, 1] = MI_std

            if save:
                np.save(f'./results/MI_stellar_spectra.npy', all_MI_estimates)

Estimating MI...


### Plot results

In [ ]:
# load pre-saved results
all_MI_estimates = np.load(f'./results/MI_stellar_spectra.npy')

In [ ]:
y_labels = ['Radial\nvelocity','T$_{\mathregular{eff}}$','Mass','[M/H]','Air\nmass','SNR']
x_labels = [f"$z_{{{i}}}$" for i in significant_dimensions]
MI_values = all_MI_estimates[:, :, 0]

In [ ]:
# and plot
im, cbar = create_heatmap(MI_values, y_labels, x_labels, cmap="YlGn", 
                          cbarlabel="MI [nats]", alpha=0.9, vmax=1.5, 
                          vmin=0, fsize=23)
texts = annotate_heatmap(im, valfmt="{x:.2f}", save_fig=True, 
                         save_path='./figures/Sedaghat_MI_1.pdf',
                         fontsize=23)

### Compare to MI values using histograms, as in Sedaghat et al. (2021)
The following code is obtained and adapted from the [astro-machines repository](https://github.com/NimSed/astro-machines).

In [ ]:
#--- A simple joint Probability Density Estimator
def pde2(d1,d2,nbins = 10, sigma=0):
    if sigma > 0:
        d1 = sigma_clip(d1,sigma=sigma)
        d2 = sigma_clip(d2,sigma=sigma)
        
    h,_,_ = np.histogram2d(d1,d2,bins=[nbins,nbins])
    h = h / np.sum(h)
    return h

#----------------------------------------------------------------------------
# Mutual information for joint histogram
# from https://matthew-brett.github.io/teaching/mutual_information.html
#----------------------------------------------------------------------------
def mutual_information(hgram): 
    # Convert bins counts to probability values
    pxy = hgram / float(np.sum(hgram))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    px_py = px[:, None] * py[None, :] # Broadcast to multiply marginals
    # Now we can do the calculation using the pxy, px_py 2D arrays
    nzs = pxy > 0 # Only non-zero pxy values contribute to the sum
    return np.sum(pxy[nzs] * np.log(pxy[nzs] / px_py[nzs]))

nDimensions = codes.shape[1]

sed_MI = np.zeros((len(significant_dimensions), nDimensions))
for k, stellar_param in enumerate(['radvel','Teff','Mass','[M/H]','airmass','snr']):
    label = labels.loc[:,stellar_param].values
    
    #-- Drop NaNs in labels
    ind_mask = ~np.isnan(label)

    #-- Ignore outlier (most probably erroneous) labels
    label_ = sigma_clip(label,sigma=5,masked=True)
    ind_mask = ind_mask & (~label_.mask)
    label_ = label_.data[ind_mask]
    print(f'# of labels: {len(label_)}')

    #--- Compute the MI indicator with 40 bins
    nbinss = [40]
    MI = np.zeros(nDimensions)
    MIs = np.zeros([len(nbinss),nDimensions])

    for b, nbins in enumerate(nbinss):
        for i in range(nDimensions):
            codes_ = codes.iloc[:,i].values
            codes_ = codes_[ind_mask]
            pqk = pde2(codes_,label_,nbins)
            MI[i] = mutual_information(pqk)
            MIs[b,i] = MI[i]
            sed_MI[k, i] = MI[i] 

# only keep significante ones            
sed_MI = sed_MI[:, significant_dimensions]

In [ ]:
# and plot
im, cbar = create_heatmap(sed_MI, y_labels, x_labels, cmap="YlGn", 
                          cbarlabel="MI [nats]", alpha=0.9, vmax=1.5, 
                          vmin=0, fsize=23)
texts = annotate_heatmap(im, valfmt="{x:.2f}", save_fig=True, 
                         save_path='./figures/Sedaghat_MI_2.pdf',
                         fontsize=23)